# Hybrid Motor


A hybrid motor is motor composed by a solid fuel grain and a liquid oxidizer.
The solid fuel grain is usually made of a rubber-like material, and the liquid
oxidizer is usually nitrous oxide. The solid fuel grain is ignited by a spark
plug, and the liquid oxidizer is injected into the combustion chamber through
an injector. The combustion of the solid fuel grain and the liquid oxidizer
generates the thrust.

For simulating this kind of motor in RocketPy, we internally use two motors:
one for the solid fuel grain and another for the liquid oxidizer. The
solid fuel grain motor is simulated as a solid motor, and the liquid oxidizer
motor is simulated as a liquid motor.

Everything is defined at the same time using the HybridMotor class. Let's
see an example:


In [ ]:
from rocketpy.motors import HybridMotor, CylindricalTank
from rocketpy.motors.Tank import MassFlowRateBasedTank
from rocketpy.motors import Fluid

## Setting the fluids


In [ ]:
oxidizer_liq = Fluid(name="N2O_l", density=1220, quality=1)
oxidizer_gas = Fluid(name="N2O_g", density=1.9277, quality=1)

## Setting the tank


The tank will hold the liquid oxidizer, in this case, nitrous oxide.

We first create te tank geometry, and then the tank object itself.


In [ ]:
tank_shape = CylindricalTank(115 / 2000, 0.705)

In [ ]:
oxidizer_tank = MassFlowRateBasedTank(
    name="oxidizer tank",
    geometry=tank_shape,
    flux_time=(5.2),
    initial_liquid_mass=4.11,
    initial_gas_mass=0,
    liquid_mass_flow_rate_in=0,
    liquid_mass_flow_rate_out=(4.11 - 0.5) / 5.2,
    gas_mass_flow_rate_in=0,
    gas_mass_flow_rate_out=0,
    liquid=oxidizer_liq,
    gas=oxidizer_gas,
)

## Create the HybridMotor object


We are using a lambda function as the thrust curve, but keep in mind that you
can use a myriad of different formats here.


In [ ]:
hybrid_motor = HybridMotor(
    thrust_source=lambda t: 2000 - (2000 - 1400) / 5.2 * t,
    dry_mass=100,
    dry_inertia=(0, 0, 0),
    center_of_dry_mass=550 / 2000 + 109 / 1000,
    burn_time=5.2,
    reshape_thrust_curve=False,
    grain_number=4,
    grain_separation=0,
    grain_outer_radius=115 / 2000,
    grain_initial_inner_radius=0.05 / 2,
    grain_initial_height=550 / 4000,
    grain_density=900,
    nozzle_radius=63.36 / 2000,
    throat_radius=26 / 2000,
    interpolation_method="linear",
    grains_center_of_mass_position=550 / 2000 + 109 / 1000,
)

## Visualize the results


At this point, you can already check the thrust curve of the motor.


In [ ]:
hybrid_motor.thrust()

However, we still need to add the tank to the motor.


In [ ]:
hybrid_motor.add_tank(
    tank=oxidizer_tank, position=2 * 550 / 2000 + 109 / 1000 + 0.705 / 2
)

Now we can proceed with several different plots.


In [ ]:
hybrid_motor.propellant_mass.plot(0, 5.2)

In [ ]:
hybrid_motor.mass_flow_rate.plot(0, 5.2)

In [ ]:
hybrid_motor.center_of_mass.plot(0, 5.2, samples=50)

We can check the inertia plots altogether, let's see how to do it by using the
`Function.compare_plots()` method.


In [ ]:
from rocketpy import Function

inertias = [
    (
        hybrid_motor.I_11,
        "I_11",
    ),
    (
        hybrid_motor.I_22,
        "I_22",
    ),
    (
        hybrid_motor.I_33,
        "I_33",
    ),
]

Function.compare_plots(
    plot_list=inertias,
    lower=0,
    upper=5.2,
    title="Inertia moments of the hybrid motor",
    xlabel="Time (s)",
    ylabel="Inertia (kg * m^2)",
)

Alternatively, one may also want to check all the information at once.
This can be done using the `all_info()` method.


In [ ]:
hybrid_motor.all_info()